# Fit Neg Binomial Auto-regressive model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial AR parameters ($\beta, \rho, \sigma^2$) for 10 iterations
    - Fix $\beta, \rho, \sigma^2$, fit $r$ using Newton for 10 iterations
    - Repeat until convergence

# Block updates

In [1]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices

function get_V(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = vec[i - 1] * ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

# simulation parameters
Random.seed!(12345)
N = 10000            # sample size
n = 5                # observations per subject
p = 3                # number of fixed effects, including intercept
σ2true = 0.1         # true σ2
ρtrue = 0.8          # true ρ
r = 100              # true dispersion parameter in NB model
β_true = ones(p)     # true beta
V = get_V(ρtrue, n)  # true variance
Γ = σ2true * V;      # true Gamma

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1317


In [2]:
# now simulate y
Random.seed!(12345)
d = NegativeBinomial()
link = LogLink()
gcs = Vector{NBCopulaARObs{Float64, typeof(d), typeof(link)}}(undef, N)
res = Vector{Float64}(undef, n)
for i in 1:N
    y = Vector{Float64}(undef, n)
    X = [ones(n) randn(n, p - 1)]
    η = X * β_true
    μ = exp.(η)
    prob = r ./ (μ .+ r)
    vecd = [NegativeBinomial(r, prob[i]) for i in 1:n]
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
    rand(nonmixed_multivariate_dist, y, res) # this mutates y!!
    gcs[i] = NBCopulaARObs(y, X, d, link)
end
gcm = NBCopulaARModel(gcs);

In [3]:
initialize_model!(gcm)
@show gcm.β
@show gcm.r
@show gcm.σ2
@show gcm.ρ;

Initializing NegBin r to Poisson regression values

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

initializing variance parameters in AR model using MM-Algorithm
gcm.β = [0.9933940020308198, 0.9994905989743817, 1.005411607019072]
gcm.r = [62.91088463799556]
gcm.σ2 = [0.05737592359284831]
gcm.ρ = [1.0]


In [4]:
# fit negative binomial AR model
@time GLMCopula.fit!(gcm, maxBlockIter=30)

Converging when tol ≤ 1.0e-7 (max block iter = 30)
Block iter 1 r = 92.58, logl = -96767.9, tol = 96767.90107993555
Block iter 2 r = 102.65, logl = -96736.08, tol = 0.00032879610898791295
Block iter 3 r = 105.56, logl = -96734.29, tol = 1.8571703899165032e-5
Block iter 4 r = 106.31, logl = -96734.17, tol = 1.2367673751417616e-6
 11.632172 seconds (9.81 M allocations: 1.362 GiB, 1.90% gc time, 3.22% compilation time)


-96734.15942486984

In [5]:
println("estimated β = $(gcm.β); true β = $β_true")
println("estimated rho = $(gcm.ρ[1]); true rho = $ρtrue")
println("estimated σ2 = $(gcm.σ2[1]); true σ2 = $σ2true")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = [1.0017847344985307, 0.9971653221090112, 1.0015724066469878]; true β = [1.0, 1.0, 1.0]
estimated rho = 0.7817532190036688; true rho = 0.8
estimated σ2 = 0.09583197525125506; true σ2 = 0.1
estimated r = 106.49749858681825; true r = 100


In [15]:
for i in 1:1000
    try
        println("\n\n Running simulation $i")
        
        Random.seed!(i)
        N = 10000            # sample size
        n = 5                # observations per subject
        p = 3                # number of fixed effects, including intercept
        σ2true = 0.1         # true σ2
        ρtrue = 0.8          # true ρ
        r = 100              # true dispersion parameter in NB model
        β_true = ones(p)     # true beta
        V = get_V(ρtrue, n)  # true variance
        Γ = σ2true * V;      # true Gamma
        
        Random.seed!(i)
        d = NegativeBinomial()
        link = LogLink()
        gcs = Vector{NBCopulaARObs{Float64, typeof(d), typeof(link)}}(undef, N)
        res = Vector{Float64}(undef, n)
        for i in 1:N
            y = Vector{Float64}(undef, n)
            X = [ones(n) randn(n, p - 1)]
            η = X * β_true
            μ = exp.(η)
            prob = r ./ (μ .+ r)
            vecd = [NegativeBinomial(r, prob[i]) for i in 1:n]
            nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
            rand(nonmixed_multivariate_dist, y, res) # this mutates y!!
            gcs[i] = NBCopulaARObs(y, X, d, link)
        end
        gcm = NBCopulaARModel(gcs)
        
        initialize_model!(gcm)
        
        @time GLMCopula.fit!(gcm, maxBlockIter=30)
        GC.gc()
    catch
        error("i = $i failed")
    end
end



 Running simulation 1
Initializing NegBin r to Poisson regression values


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
Converging when tol ≤ 1.0e-7 (max block iter = 30)
Block iter 1 r = 84.63, logl = -96778.59, tol = 96778.59239652216
Block iter 2 r = 91.38, logl = -96755.38, tol = 0.00023983148408301345
Block iter 3 r = 93.07, logl = -96754.29, tol = 1.1307010576870748e-5
Block iter 4 r = 93.37, logl = -96754.14, tol = 1.482529917427237e-6
 10.152201 seconds (9.24 M allocations: 1.325 GiB, 3.23% gc time)


 Running simulation 2
Initializing NegBin r to Poisson regression values


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
Converging when tol ≤ 1.0e-7 (max block iter = 30)
Block iter 1 r = 92.9, logl = -96635.94, tol = 96635.94287716773
Block iter 2 r = 102.74, logl = -96602.37, tol = 0.000347371118945161
Block iter 3 r = 105.31, logl = -96600.66, tol = 1.7712267133429735e-5
Block iter 4 r = 105.69, logl = -96600.58, tol = 8.080327914840375e-7
Block iter 5 r = 105.83, logl = -96600.62, tol = 3.347308962634572e-7
Block iter 6 r = 106.06, logl = -96600.56, tol = 5.784584257220894e-7
 13.936419 seconds (12.90 M allocations: 1.851 GiB, 2.16% gc time)


 Running simulation 3
Initializing NegBin r to Poisson regression values


...

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
Converging when tol ≤ 1.0e-7 (max block iter = 30)
Block iter 1 r = 79.31, logl = -97039.08, tol = 97039.08414434122
Block iter 2 r = 85.96, logl = -97017.5, tol = 0.0002224425908604809
Block iter 3 r = 87.91, logl = -97016.12, tol = 1.4201429231755863e-5
Block iter 4 r = 88.43, logl = -97016.02, tol = 1.0252300763670215e-6
 11.049118 seconds (9.24 M allocations: 1.325 GiB, 2.33% gc time)


 Running simulation 81
Initializing NegBin r to Poisson regression values


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
Converging when tol ≤ 1.0e-7 (max block iter = 30)
Block iter 1 r = 86.27, logl = -97301.0, tol = 97300.9964817039
Block iter 2 r = 94.83, logl = -97270.92, tol = 0.00030913842658913753
Block iter 3 r = 97.02, logl = -97269.14, tol = 1.827362091892653e-5
Block iter 4 r = 97.85, logl = -97269.02, tol = 1.2240824979101507e-6
Block iter 5 r = 97.95, logl = -97269.01, tol = 1.1219403081977211e-7


LoadError: i = 81 failed

## Debug $r < 0$ case

In [7]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices
function run_test()
    p_fixed = 3    # number of fixed effects, including intercept
    # true parameter values
    βtrue = ones(p_fixed)
    σ2true = [0.1]
    ρtrue = [0.9]
    rtrue = 10.0

    function get_V(ρ, n)
        vec = zeros(n)
        vec[1] = 1.0
        for i in 2:n
            vec[i] = vec[i - 1] * ρ
        end
        V = ToeplitzMatrices.SymmetricToeplitz(vec)
        V
    end

    # generate data
    intervals = zeros(p_fixed + 3, 2) #hold intervals
    curcoverage = zeros(p_fixed + 3) #hold current coverage resutls
    trueparams = [βtrue; ρtrue; σ2true; rtrue] #hold true parameters

    #simulation parameters
    samplesizes = [100; 1000; 10000]
    ns = [5; 10; 15; 20; 25]
    nsims = 5

    #storage for our results
    βMseResults = ones(nsims * length(ns) * length(samplesizes))
    σ2MseResults = ones(nsims * length(ns) * length(samplesizes))
    ρMseResults = ones(nsims * length(ns) * length(samplesizes))
    rMseResults = ones(nsims * length(ns) * length(samplesizes))

    βρσ2rcoverage = Matrix{Float64}(undef, p_fixed + 3, nsims * length(ns) * length(samplesizes))
    fittimes = zeros(nsims * length(ns) * length(samplesizes))

    solver = Ipopt.IpoptSolver(print_level = 5)

    st = time()
    currentind = 1
    d = NegativeBinomial()
    link = LogLink()
    D = typeof(d)
    Link = typeof(link)
    T = Float64

    for t in 1:length(samplesizes)
        m = samplesizes[t]
        gcs = Vector{NBCopulaARObs{T, D, Link}}(undef, m)
        for k in 1:length(ns)
            ni = ns[k] # number of observations per individual
            V = get_V(ρtrue[1], ni)
            # true Gamma
            Γ = σ2true[1] * V

            for j in 1:nsims
                println("rep $j obs per person $ni samplesize $m")
                Y_nsample = []
                for i in 1:m
                    Random.seed!(1000000000 * t + 10000000 * j + 1000000 * k + i)
                    X = [ones(ni) randn(ni, p_fixed - 1)]
                    η = X * βtrue
                    μ = exp.(η)
                    p = rtrue ./ (μ .+ rtrue)
                    vecd = Vector{DiscreteUnivariateDistribution}(undef, ni)
                    vecd = [NegativeBinomial(rtrue, p[i]) for i in 1:ni]
                    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
                    # simuate single vector y
                    y = Vector{Float64}(undef, ni)
                    res = Vector{Float64}(undef, ni)
                    # Random.seed!(1000000000 * t + 10000000 * j + 1000000 * k + i)
                    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
                    # simuate single vector y
                    y = Vector{Float64}(undef, ni)
                    res = Vector{Float64}(undef, ni)
                    Random.seed!(1000000000 * t + 10000000 * j + 1000000 * k + i)
                    rand(nonmixed_multivariate_dist, y, res)
                    gcs[i] = NBCopulaARObs(y, X, d,link)
                    push!(Y_nsample, y)
                end

                # form model
                gcm = NBCopulaARModel(gcs);
                fittime = NaN
                initialize_model!(gcm)
                @show gcm.β
                @show gcm.ρ
                @show gcm.σ2
                @show gcm.r

                # ### now sigma2 is initialized now we need rho
#                 Y_1 = [Y_nsample[i][1] for i in 1:m]
#                 Y_2 = [Y_nsample[i][2] for i in 1:m]

#                 update_rho!(gcm, Y_1, Y_2)
                # @show gcm.ρ
                # @show gcm.σ2
                try
                    fittime = @elapsed GLMCopula.fit!(gcm)
#                     fittime = @elapsed GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, tol = 10^-8, limited_memory_max_history = 20, accept_after_max_steps = 1, hessian_approximation = "limited-memory"))
                    # fittime = @elapsed GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, tol = 10^-5, hessian_approximation = "limited-memory"))
                    @show fittime
                    @show gcm.θ
                    @show gcm.∇θ
                    @show gcm.r
                    @show gcm.∇r
                    loglikelihood!(gcm, true, true)
                    vcov!(gcm)
                    @show GLMCopula.confint(gcm)

                # mse and time under our model
                # coverage!(gcm, trueparams, intervals, curcoverage)
                mseβ, mseρ, mseσ2, mser = MSE(gcm, βtrue, ρtrue, σ2true, rtrue)
                @show mseβ
                @show mser
                @show mseσ2
                @show mseρ
                # global currentind
                @views copyto!(βρσ2rcoverage[:, currentind], curcoverage)
                βMseResults[currentind] = mseβ
                rMseResults[currentind] = mser
                σ2MseResults[currentind] = mseσ2
                ρMseResults[currentind] = mseρ
                fittimes[currentind] = fittime
                currentind += 1

                catch
                    println("rep $j ni obs = $ni , samplesize = $m had an error")
                    βMseResults[currentind] = NaN
                    rMseResults[currentind] = NaN
                    σ2MseResults[currentind] = NaN
                    ρMseResults[currentind] = NaN
                    βρσ2rcoverage[:, currentind] .= NaN
                    fittimes[currentind] = NaN
                    currentind += 1
                 end
            end
        end
    end
end
run_test()


rep 1 obs per person 5 samplesize 100
Initializing NegBin r to Poisson regression values
initializing variance parameters in AR model using MM-Algorithm
gcm.β = [0.8300033992875536, 1.1316939239092045, 1.11158300966302]
gcm.ρ = [1.0]
gcm.σ2 = [0.16950409437121006]
gcm.r = [10.971850330351796]
Converging when tol ≤ 1.0e-7 (max block iter = 100)
Block iter 1 r = 10.78, logl = -1025.57, tol = 1025.5666961068969
Block iter 2 r = 10.75, logl = -1025.56, tol = 6.965955721159785e-6
Block iter 3 r = 10.75, logl = -1025.56, tol = 1.7176739203244544e-7
fittime = 0.16093157
gcm.θ = [0.9701929641512187, 1.013137287252717, 1.0473073172734642, 1.0, 0.10494299897469216]
gcm.∇θ = [-0.0038246346048609325, 0.00758767494764756, 0.010442328116452515, 0.2639351744946713, 0.005814215122279309]
gcm.r = [10.74505965670014]
gcm.∇r = [-1.2929883153767197e-5]
GLMCopula.confint(gcm) = [0.9141365608335388 1.0262493674688986; 0.9329208531628778 1.093353721342556; 0.9877394202687937 1.1068752142781346; 0.63953670034

┌ Warning: Ipopt finished with status Restoration_Failed
└ @ Ipopt /Users/biona001/.julia/packages/Ipopt/QF8Lc/src/MPB_wrapper.jl:195
┌ Warning: Optimization unsuccesful; got Error
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31
┌ Warning: Ipopt finished with status Restoration_Failed
└ @ Ipopt /Users/biona001/.julia/packages/Ipopt/QF8Lc/src/MPB_wrapper.jl:195


Block iter 1 r = 5.06, logl = -4643.54, tol = 4643.543292480797
Block iter 2 r = 9.09, logl = -4139.61, tol = 0.1085010173297323
Block iter 3 r = 9.42, logl = -4102.89, tol = 0.008867426007071381
Block iter 4 r = 9.44, logl = -4102.77, tol = 2.954578676015983e-5
fittime = 0.33738419
gcm.θ = [0.9388921032985122, 1.0555137549802895, 1.031032633981741, 0.7958046307362826, 4953.858335378658]
gcm.∇θ = [0.0030685472608942277, 0.00029529995095955996, -0.0043440135306958005, 0.0003829983253209601, -3.659696003917843e-8]
gcm.r = [9.437511176898258]
gcm.∇r = [5.413597519252772e-5]
GLMCopula.confint(gcm) = [0.8985277496245413 0.9792564569724832; 1.0346146324728087 1.0764128774877704; 1.012852644181105 1.0492126237823771; 0.6519994881402409 0.9396097733323243; 1267.225697553241 8640.490973204076; 8.24653048387384 10.628491869922676]
mseβ = 0.0025929921343786763
mser = 0.3163936761143823
mseσ2 = 2.453972164533353e7
mseρ = 0.010856674976002428
rep 5 obs per person 20 samplesize 100
Initializing NegB

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


gcm.β = [-0.5831714019428005, 0.6352087955114383, 0.8278861167443692]
gcm.ρ = [1.0]
gcm.σ2 = [9638.682810328937]
gcm.r = [1.0]
Converging when tol ≤ 1.0e-7 (max block iter = 100)
Block iter 1 r = 2.35, logl = -12106.0, tol = 12106.00434699679
Block iter 2 r = 5.07, logl = -11363.93, tol = 0.06129265831015232
Block iter 3 r = 11.3, logl = -10907.14, tol = 0.04019304114291532
Block iter 4 r = 13.28, logl = -10680.59, tol = 0.02076895067807495
Block iter 5 r = 13.55, logl = -10673.25, tol = 0.0006870286084499532
Block iter 6 r = 13.58, logl = -10673.14, tol = 1.0289826883780855e-5
Block iter 7 r = 13.58, logl = -10673.14, tol = 1.1166934022200233e-7
fittime = 1.197134194
gcm.θ = [0.9041933993512293, 1.052797952850866, 1.039069803127839, 0.2842442975797953, 9638.68290324939]
gcm.∇θ = [0.17215045368871085, -0.4216306678732291, -0.1621413788539856, 0.19704852992343247, -1.8368296696541465e-6]
gcm.r = [13.585061058621053]
gcm.∇r = [0.0002016226447944386]
GLMCopula.confint(gcm) = [0.8888484469

┌ Warning: Ipopt finished with status Restoration_Failed
└ @ Ipopt /Users/biona001/.julia/packages/Ipopt/QF8Lc/src/MPB_wrapper.jl:195
┌ Warning: Optimization unsuccesful; got Error
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31
┌ Warning: Ipopt finished with status Restoration_Failed
└ @ Ipopt /Users/biona001/.julia/packages/Ipopt/QF8Lc/src/MPB_wrapper.jl:195


gcm.β = [-0.028089692833428267, 0.013901471690725116, 0.0038290446952773667]
gcm.ρ = [1.0]
gcm.σ2 = [3982.3602132796505]
gcm.r = [1.0]
Converging when tol ≤ 1.0e-7 (max block iter = 100)
Block iter 1 r = 5.82, logl = -583873.22, tol = 583873.2176419455
Block iter 2 r = 8.88, logl = -518485.22, tol = 0.11198986901051508
Block iter 3 r = 9.05, logl = -516161.71, tol = 0.004481341178927795
Block iter 4 r = 9.06, logl = -516157.22, tol = 8.683438718461431e-6
fittime = 28.446139321
gcm.θ = [0.9672941110976758, 1.014095766000179, 1.01720771567053, 0.7887054558152341, 3982.360245762748]
gcm.∇θ = [1.5856939426295389, 1.4910481584911883, 2.117263660357829, -0.27173941113834044, -5.92421655850532e-6]
gcm.r = [9.056992770421902]
gcm.∇r = [0.0019619205600797524]
GLMCopula.confint(gcm) = [0.9646599761145135 0.9699282460808382; 1.01105991986347 1.017131612136888; 1.0143156795162847 1.0200997518247752; 0.7711354347392849 0.8062754768911833; 3735.23634385265 4229.484147672846; 8.950037441712889 9.1639